# 示例：从新闻网站点击流中挖掘

## FP-growth算法

In [1]:
# 创建FP树的数据结构
class treeNode:
    # nameValue: 节点名字，numOccur: 出现次数，parentNode：父节点
    def __init__(self, nameValue, numOccur, parentNode):
        # 节点名字
        self.name = nameValue
        # 计数值
        self.count = numOccur
        # 链接相似的元素项
        self.nodeLink = None
        # 指向当前节点的父节点
        self.parent = parentNode
        # 存放节点的子节点
        self.children = {} 
    
    # 对count增加给定值
    def inc(self, numOccur):
        self.count += numOccur
        
    # 将树以文本形式展示
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
# 构建FP树主函数
# dataSet：字典，其中项集为字典的键，值为频率，minSup：最小支持度
def createTree(dataSet, minSup=1):
    # 头指针表
    headerTable = {}
    # 第一次遍历：统计每个元素项出现的频度
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    headerTableCopy = headerTable.copy()
    # 扫描头指针表，删掉出现次数小于最小支持度的项
    for k in headerTableCopy.keys():
        if headerTable[k] < minSup: 
            del(headerTable[k])
    # 得到频繁项集
    freqItemSet = set(headerTable.keys())
    # 如果所有项都不频繁，就不需要进行下一步处理
    if len(freqItemSet) == 0: return None, None
    # 对头指针表进行扩展，以便可以保存计数值以及指向每种类型第一个元素项的指针
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] 
    # 先创建仅包含空集合的根节点
    retTree = treeNode('Null Set', 1, None)
    # 第二次遍历：构建树
    for tranSet, count in dataSet.items():
        localD = {}
        # 根据全局频率对每个事务中的元素进行排序
        for item in tranSet:
            # 只考虑频繁项集
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            # 使用排序后的频率项集对树进行填充
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

# 填充树函数
# items: 一个项集，inTree: FP树，headerTable: 头指针表，count：频率数
def updateTree(items, inTree, headerTable, count):
    # 事务中的第一个元素项是否作为子节点存在
    if items[0] in inTree.children:
        # 更新该元素项的计数
        inTree.children[items[0]].inc(count)
    else:
        # 如果不存在，创建一个新的treeNode并将其作为一个子节点添加的树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        # 更新头指针表
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:
        # 迭代，每次调用都去掉列表中第一个元素
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 更新头指针表
def updateHeader(nodeToTest, targetNode):
    # 从头指针表的nodeLink开始，一直沿着nodeLink直到到达链表末尾
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    # 最后再添加最后一个节点
    nodeToTest.nodeLink = targetNode

In [3]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [4]:
# 构建前缀路径，迭代上溯整棵树
def ascendTree(leafNode, prefixPath): 
    if leafNode.parent != None:
        # 加入前缀路径
        prefixPath.append(leafNode.name)
        # 继续上溯迭代
        ascendTree(leafNode.parent, prefixPath)

# 前缀路径
# basePat: 条件模式基，treeNode: 树节点-头指针表
def findPrefixPath(basePat, treeNode): 
    condPats = {}
    while treeNode != None:
        prefixPath = []
        # 得到前缀路径
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            # 去掉所查元素，构建条件模式基
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        # 遍历链表中的下一个节点
        treeNode = treeNode.nodeLink
    return condPats

In [5]:
# 递归查找频繁项集的mineTree函数
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 对头指针表中的元素项按照出现频率排序
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]
    # 从头指针表的底端开始
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        # 创建条件基
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        # 从条件模式基来构建条件FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
        # 如果树中还有元素项，递归调用mineTree()函数
        if myHead != None:
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

## 将数据集导入到列表中

In [6]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]

## 对初始集合进行格式化

In [7]:
initSet = createInitSet(parsedDat)

## 构建FP树，并从中寻找那些至少被10万人浏览过的新闻报道

In [8]:
myFPtree, myHeaderTab = createTree(initSet, 100000)

## 创建一个空列表来保存这些频繁项集

In [9]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

conditional tree for:  {'1'}
   Null Set   1
     6   107404
conditional tree for:  {'11'}
   Null Set   1
     6   261773
conditional tree for:  {'3'}
   Null Set   1
     6   186289
       11   117401
     11   9718
conditional tree for:  {'11', '3'}
   Null Set   1
     6   117401


In [10]:
len(myFreqList)

9

In [11]:
myFreqList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'6'}]